In [59]:
import os
import pandas as pd
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from IPython.display import display

In [60]:
COMMODITY = 'magnesium'
METRIC = 'Recall'
ALGO = 'detection_correction'
# ALGO = 'correction'


EDCR_RESULTS_PATH = f'{COMMODITY}/test/predictions/test_{METRIC}_{ALGO}'
OG_RESULTS_PATH = f'{COMMODITY}/test/results_test.csv'

In [61]:
def extract_info_from_filename(filename):
    parts = filename.split("Rule")
    epsilon = parts[0].split()[-1]
    models_base = parts[1].split("for")
    models = models_base[0].strip()
    base = models_base[1].split("_predictions")[0].strip()
    return base, models, epsilon

In [62]:
def calculate_metrics(df):
    y_true = df['True']
    y_pred = df['Predicted']
    accuracy = accuracy_score(y_true, y_pred)
    precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average=None, labels=[0, 1])
    prior = pd.read_csv(OG_RESULTS_PATH, nrows=1)['Prior'].iloc[0]
    return accuracy, precision, recall, f1, prior

In [63]:
def process_files(path):
    all_results = []  
    for filename in os.listdir(path):
        if filename.startswith('Confident') and filename.endswith("_predictions.csv"):
            base, models, confidence = extract_info_from_filename(filename)
            df = pd.read_csv(os.path.join(path, filename))
            accuracy, precision, recall, f1, prior = calculate_metrics(df)
            row = {
                'Base': base,
                'Models': models,
                'Confidence': confidence,
                'Accuracy': accuracy,
                'Precision (0)': precision[0],
                'Recall (0)': recall[0],
                'F1 (0)': f1[0],
                'Precision (1)': precision[1],
                'Recall (1)': recall[1],
                'F1 (1)': f1[1],
                'Prior': prior  
            }
            all_results.append(row)
    
    # Convert the list of results to a DataFrame
    return pd.DataFrame(all_results)

In [64]:
results_df = process_files(EDCR_RESULTS_PATH)

c:\Users\manim\LabV2\ONR-Price-Prediction\onr_price_prediction\venv\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\manim\LabV2\ONR-Price-Prediction\onr_price_prediction\venv\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\manim\LabV2\ONR-Price-Prediction\onr_price_prediction\venv\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to cont

In [65]:
results_df_one = results_df[results_df['Models'] != 'all']
# pd.set_option('display.max_rows', 200)
results_df_one

,Base,Models,Confidence,Accuracy,Precision (0),Recall (0),F1 (0),Precision (1),Recall (1),F1 (1),Prior
34,CNN_128_7,CNNA_128_3,0.1,0.2,0.0,0.0,0.000000,0.2,1.0,0.333333,0.2
35,CNN_256_3,CNNA_128_3,0.1,0.2,0.0,0.0,0.000000,0.2,1.0,0.333333,0.2
36,CNN_256_5,CNNA_128_3,0.1,0.2,0.0,0.0,0.000000,0.2,1.0,0.333333,0.2
37,CNN_256_7,CNNA_128_3,0.1,0.2,0.0,0.0,0.000000,0.2,1.0,0.333333,0.2
38,CNN_32_3,CNNA_128_3,0.1,0.2,0.0,0.0,0.000000,0.2,1.0,0.333333,0.2
...,...,...,...,...,...,...,...,...,...,...,...
2323,RNN_128,RNN_32,0.9,0.8,0.8,1.0,0.888889,0.0,0.0,0.000000,0.2
2324,RNN_256,RNN_32,0.9,0.8,0.8,1.0,0.888889,0.0,0.0,0.000000,0.2
2325,RNN_32,RNN_32,0.9,0.8,0.8,1.0,0.888889,0.0,0.0,0.000000,0.2
2326,RNN_64,RNN_32,0.9,0.8,0.8,1.0,0.888889,0.0,0.0,0.000000,0.2


In [66]:
og_df = pd.read_csv(OG_RESULTS_PATH)
og_df = og_df.drop('Unnamed: 0', axis=1)
og_df = og_df[(og_df['Name'] != 'EDCR')]
pd.set_option('display.max_rows', 400)
og_df

,Name,Params,Accuracy,Precision (0),Recall (0),F1 (0),Precision (1),Recall (1),F1 (1),Prior
0,LSTM,256 layers,0.274419,0.785714,0.127907,0.220000,0.197861,0.860465,0.321739,0.2
1,LSTM,128 layers,0.320930,0.842105,0.186047,0.304762,0.209040,0.860465,0.336364,0.2
2,LSTM,64 layers,0.204651,0.545455,0.034884,0.065574,0.186275,0.883721,0.307692,0.2
3,LSTM,32 layers,0.218605,0.625000,0.058140,0.106383,0.185930,0.860465,0.305785,0.2
4,CNN with Attention,"32 filters, kernel size 7",0.339535,0.826087,0.220930,0.348624,0.207101,0.813953,0.330189,0.2
5,CNN with Attention,"32 filters, kernel size 5",0.306977,0.828571,0.168605,0.280193,0.205556,0.860465,0.331839,0.2
6,CNN with Attention,"32 filters, kernel size 3",0.786047,0.797170,0.982558,0.880208,0.000000,0.000000,0.000000,0.2
7,CNN with Attention,"64 filters, kernel size 7",0.386047,0.870370,0.273256,0.415929,0.223602,0.837209,0.352941,0.2
8,CNN with Attention,"64 filters, kernel size 5",0.441860,0.893939,0.343023,0.495798,0.241611,0.837209,0.375000,0.2
9,CNN with Attention,"64 filters, kernel size 3",0.460465,0.868421,0.383721,0.532258,0.237410,0.767442,0.362637,0.2


### Base Model

Generally for lower confidence (0.1-0.3), the rule is basically all '1'. It's always predicting spikes, so that's why lower confidence results look similar to spikes-dumb-model

Conversely, for higher confidence (0.9-0.95), the rule is becoming more like all '0'. So the higher we go, the less spikes we're predicting, hence making it look more like non-spikes-dumb-model

In [67]:
MODEL_NAME_OF_INTEREST = 'CNN'
MODEL_PARAM_OF_INTEREST = '32_5'

results_df_all = results_df[(results_df['Base'] == f'{MODEL_NAME_OF_INTEREST}_{MODEL_PARAM_OF_INTEREST}') & (results_df['Models'] == 'all') & (results_df['Base'] != "dumb_spikes") & (results_df['Base'] != "dumb_non_spikes")]
results_df_all

,Base,Models,Confidence,Accuracy,Precision (0),Recall (0),F1 (0),Precision (1),Recall (1),F1 (1),Prior
19,CNN_32_5,all,0.1,0.200000,0.000000,0.000000,0.000000,0.200000,1.000000,0.333333,0.2
213,CNN_32_5,all,0.2,0.200000,0.000000,0.000000,0.000000,0.200000,1.000000,0.333333,0.2
407,CNN_32_5,all,0.3,0.200000,0.000000,0.000000,0.000000,0.200000,1.000000,0.333333,0.2
601,CNN_32_5,all,0.45,0.200000,0.000000,0.000000,0.000000,0.200000,1.000000,0.333333,0.2
795,CNN_32_5,all,0.4,0.200000,0.000000,0.000000,0.000000,0.200000,1.000000,0.333333,0.2
989,CNN_32_5,all,0.55,0.325581,0.846154,0.191860,0.312796,0.210227,0.860465,0.337900,0.2
1183,CNN_32_5,all,0.5,0.204651,0.555556,0.029070,0.055249,0.189320,0.906977,0.313253,0.2
1377,CNN_32_5,all,0.6,0.386047,0.884615,0.267442,0.410714,0.226994,0.860465,0.359223,0.2
1571,CNN_32_5,all,0.7,0.502326,0.873563,0.441860,0.586873,0.250000,0.744186,0.374269,0.2
1765,CNN_32_5,all,0.8,0.679070,0.841060,0.738372,0.786378,0.296875,0.441860,0.355140,0.2


### Dumb Model (Only Predicts Non-Spikes)

In [68]:
results_df_all = results_df[(results_df['Models'] == 'all') & (results_df['Base'] == "dumb_non_spikes")]
results_df_all

,Base,Models,Confidence,Accuracy,Precision (0),Recall (0),F1 (0),Precision (1),Recall (1),F1 (1),Prior
24,dumb_non_spikes,all,0.1,0.200000,0.000000,0.000000,0.000000,0.200000,1.000000,0.333333,0.2
218,dumb_non_spikes,all,0.2,0.200000,0.000000,0.000000,0.000000,0.200000,1.000000,0.333333,0.2
412,dumb_non_spikes,all,0.3,0.200000,0.000000,0.000000,0.000000,0.200000,1.000000,0.333333,0.2
606,dumb_non_spikes,all,0.45,0.200000,0.000000,0.000000,0.000000,0.200000,1.000000,0.333333,0.2
800,dumb_non_spikes,all,0.4,0.200000,0.000000,0.000000,0.000000,0.200000,1.000000,0.333333,0.2
994,dumb_non_spikes,all,0.55,0.316279,0.837838,0.180233,0.296651,0.207865,0.860465,0.334842,0.2
1188,dumb_non_spikes,all,0.5,0.204651,0.555556,0.029070,0.055249,0.189320,0.906977,0.313253,0.2
1382,dumb_non_spikes,all,0.6,0.386047,0.884615,0.267442,0.410714,0.226994,0.860465,0.359223,0.2
1576,dumb_non_spikes,all,0.7,0.502326,0.873563,0.441860,0.586873,0.250000,0.744186,0.374269,0.2
1770,dumb_non_spikes,all,0.8,0.679070,0.841060,0.738372,0.786378,0.296875,0.441860,0.355140,0.2
